In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
# import lib.zigzag as zz
# import lib.bars as bars
# from lib.FeaturesExtractors.Base import Base

In [3]:
data = pd.read_csv("..\Datasets\WDO@_M5-with-zz.csv")
# columns = ['time', 'open', 'high', 'low', 'close']
columns = ['time', 'open', 'high', 'low', 'close', 'zz', 'highs', 'lows']
data = data.loc[:, columns]
# data['time']=pd.to_datetime(data['time'], unit='ms')
data.tail()

,time,open,high,low,close,zz,highs,lows
96519,2021-04-05 17:35:00,5677.5,5685.0,5677.5,5685.0,NaN,NaN,NaN
96520,2021-04-05 17:40:00,5685.0,5685.5,5682.5,5684.0,NaN,NaN,NaN
96521,2021-04-05 17:45:00,5684.5,5686.0,5676.5,5679.5,5686.0,5686.0,NaN
96522,2021-04-05 17:50:00,5679.0,5680.5,5676.0,5677.0,5676.0,NaN,5676.0
96523,2021-04-05 17:55:00,5676.5,5679.0,5672.0,5672.0,NaN,NaN,NaN


In [4]:
# parsed_ds=data.loc[(data['time']>='2021-03-19 8:00:00')&(data['time']<='2021-03-19 19:00:00')]
parsed_ds=data.loc[(data['time']>='2021-03-22 09:00:00')&(data['time']<='2021-03-22 19:00:00')]
parsed_ds.reset_index(inplace=True, drop=True)
parsed_ds.head()

,time,open,high,low,close,zz,highs,lows
0,2021-03-22 09:00:00,5510.0,5518.0,5507.0,5516.0,NaN,NaN,NaN
1,2021-03-22 09:05:00,5516.5,5519.0,5511.0,5514.5,5519.0,5519.0,NaN
2,2021-03-22 09:10:00,5515.0,5515.5,5500.5,5500.5,NaN,NaN,NaN
3,2021-03-22 09:15:00,5500.5,5504.0,5498.0,5501.5,NaN,NaN,NaN
4,2021-03-22 09:20:00,5501.0,5513.0,5497.0,5510.5,5497.0,NaN,5497.0


In [17]:
# len(parsed_ds)
parsed_ds.shape

(108, 8)

In [5]:
fig = go.Figure(data=[go.Candlestick(open=parsed_ds['open'],
                high=parsed_ds['high'],
                low=parsed_ds['low'],
                close=parsed_ds['close'])])

fig.add_trace(go.Scatter(
    x=parsed_ds.loc[(parsed_ds['zz'].notnull())].index,
    y=parsed_ds.loc[(parsed_ds['zz'].notnull())]['zz']
))


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[19, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [327]:
'''
Cria uma Series com o inicio de cada perna
contando que cada perna começa na primeira
barra com direção igual ao ZigZag ou doji.

A perna na coluna leg_start pode 
terminar uma linha antes exluindo a ultima 
barra da perna. Logo a maneira correta de 
achar a perna completa é verificar da primeira
barra da perna até a próxima não nula ocorrencia
da coluna "zz".

É preciso saber o começo e o fim da perna para
extrair a features
'''
fls=FindLegStart()
parsed_ds=parsed_ds.assign(leg_start=fls.run)
parsed_ds.head(15)

,time,open,high,low,close,zz,highs,lows,leg_start
0,2021-01-11 09:00:00,5457.0,5462.5,5455.5,5458.5,NaN,NaN,NaN,NaN
1,2021-01-11 09:05:00,5458.5,5476.0,5453.0,5473.5,NaN,NaN,NaN,NaN
2,2021-01-11 09:10:00,5473.0,5490.5,5472.0,5489.5,NaN,NaN,NaN,NaN
3,2021-01-11 09:15:00,5489.5,5498.0,5488.0,5496.5,NaN,NaN,NaN,NaN
4,2021-01-11 09:20:00,5496.5,5499.0,5493.5,5497.5,NaN,NaN,NaN,NaN
5,2021-01-11 09:25:00,5497.0,5503.0,5489.5,5489.5,5503.0,5503.0,NaN,1.0
6,2021-01-11 09:30:00,5489.5,5494.5,5486.5,5489.5,NaN,NaN,NaN,NaN
7,2021-01-11 09:35:00,5490.0,5494.5,5485.0,5489.0,NaN,NaN,NaN,NaN
8,2021-01-11 09:40:00,5488.5,5490.5,5483.0,5484.5,NaN,NaN,NaN,NaN
9,2021-01-11 09:45:00,5484.5,5492.0,5478.0,5488.0,5478.0,NaN,5478.0,2.0


In [328]:
print("Number of legs: ",parsed_ds.loc[(parsed_ds['zz'].notnull())]['zz'].shape[0]-1)

Number of legs:  1290


In [329]:
al=AbstractLegs()
legb=al.run(parsed_ds)
legb.shape

(1282,)

In [10]:
'''
FEATURE: Contagem de barras
Diferença entre leg_sart_idx and leg_end_idx


'''

class leg_reversal_attempts(Base):
    def __init__(self) -> None:
        super().__init__()
        self.counter = 0
        self.reversed=False
        self.reversed_count=0
        self.leg_ended=False

    def loop(self, row):        
        if self.leg_end(row):
            self.leg_start_idx = None
            self.leg_ended=True            

        if self.leg_start(row):
            self.leg_start_idx = row.name

        if self.leg_start_idx!=None and self.zz_dir != None:
            bar_direction=bars.bar_direction(row)
            if bar_direction == self.zz_dir:
                self.counter += 1
            if self.counter>0 and bar_direction!=self.zz_dir:
                self.counter=0
                bar_direction=None
                self.reversed_count+=1
        
        if self.leg_ended:
            self.leg_ended=False
            return self.result()

    def result(self):
        tmp = self.reversed_count
        self.reversed_count = 0
        return tmp

        

In [11]:
l=leg_reversal_attempts()
random_variable=parsed_ds.apply(lambda row: l.loop(row), axis=1)
random_variable=random_variable.loc[~np.isnan(random_variable)]
# print()
random_variable[:30]

4      1.0
11     1.0
16     1.0
21     0.0
29     2.0
31     1.0
36     2.0
44     0.0
47     0.0
54     1.0
61     2.0
66     1.0
72     2.0
77     1.0
84     2.0
87     1.0
99     3.0
107    2.0
111    1.0
120    2.0
122    0.0
124    1.0
128    0.0
134    2.0
138    0.0
146    2.0
152    2.0
156    0.0
165    2.0
168    1.0
dtype: float64

In [42]:
fig = go.Figure(data=[go.Candlestick(x=parsed_ds['time'],
                open=parsed_ds['open'],
                high=parsed_ds['high'],
                low=parsed_ds['low'],
                close=parsed_ds['close'])])

# fig.add_trace(go.Scatter(
#     x=data.loc[(data['zz'].notnull())]['time'],
#     y=data.loc[(data['zz'].notnull())]['zz']
# ))

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[18, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [6]:
# Perna total expandida comparada com
# a perna como ela aparece no grafico.
# Relação continua de sobreposição entre 
# as barras.

contracted_leg = parsed_ds.iloc[parsed_ds.shape[0]-1]['high']-parsed_ds.iloc[0]['low']
expanded_leg=0
for idx in range(parsed_ds.shape[0]):
    expanded_leg+=parsed_ds.iloc[idx]['high']-parsed_ds.iloc[idx]['low']

print('expanded_leg: {}, contracted_leg: {}'.format(expanded_leg,contracted_leg))

expanded_leg: 93.0, contracted_leg: 32.5


In [7]:
leg_direction = -1

In [8]:
'''
Bars that trade below 50% of the bar before
'''

overlap_count=0
for idx in range(data.shape[0]):
    if idx <= 1:
        continue
    fifty=data.iloc[idx-1]['low']+(data.iloc[idx-1]['high']-data.iloc[idx-1]['low'])/2
    if leg_direction > 0 and fifty > data.iloc[idx]['low']:
        overlap_count+=1
    if leg_direction < 0 and fifty < data.iloc[idx]['high']:
        overlap_count+=1
    
    # print(idx)


In [18]:
def between(value, bar, body=True):
    direction = bars.bar_direction(bar)
    if body:
        if direction>0:
            if bar['close']>value and bar['open'] < value:
                return True
        if direction<0:
            if bar['close']<value and bar['open'] > value:
                return True
    else:
        if bar['high']>value and bar['low'] < value:
            return True
    return False

def inside_bar(outside, inside, inside_body=False):
    if not inside_body:
        if outside['high']>=inside['high'] and outside['low']<=inside['low']:
            return True
    else:
        direction=bars.bar_direction(inside)
        if direction>0:
            if outside['high']>=inside['close'] and outside['low']<=inside['open']:
                return True
        if direction<0:
            if outside['high']>=inside['open'] and outside['low']<=inside['close']:
                return True
    return False

In [57]:
inside_bar(parsed_ds.iloc[6],parsed_ds.iloc[7]) or \
inside_bar(parsed_ds.iloc[6],parsed_ds.iloc[7], inside_body=True) or \
between(parsed_ds.iloc[6]['close'],parsed_ds.iloc[7]) or \
between(parsed_ds.iloc[6]['open'],parsed_ds.iloc[7])

True

In [63]:
'''
Bars that overlap with the body of bars before
- The first bar of the range does not count
- The body overlaps with two bars before or more
'''

overlaping_count=bar_before=last_overlaping=0
range_starts=[]
while bar_before<parsed_ds.shape[0]-2:
    for overlap_bar in range(bar_before+2, parsed_ds.shape[0]):
        doji=bars.doji(parsed_ds.iloc[bar_before])
        if doji:
            overlap_flag = inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar]) or \
                        inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar], inside_body=True) or \
                        between(parsed_ds.iloc[bar_before]['high'],parsed_ds.iloc[overlap_bar]) or \
                        between(parsed_ds.iloc[bar_before]['low'],parsed_ds.iloc[overlap_bar])
            if overlap_bar == bar_before+2:
                if not overlap_flag:
                    break
            if overlap_flag:
                overlaping_count+=1
                last_overlaping=overlap_bar
                # print('bar_before: ', bar_before)
        if not doji:
            overlap_flag=inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar]) or \
                    inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar], inside_body=True) or \
                    between(parsed_ds.iloc[bar_before]['close'],parsed_ds.iloc[overlap_bar]) or \
                    between(parsed_ds.iloc[bar_before]['open'],parsed_ds.iloc[overlap_bar])
            if overlap_bar == bar_before+2:
                if not overlap_flag:
                    break
            if overlap_flag:
                overlaping_count+=1
                last_overlaping=overlap_bar

    if last_overlaping>bar_before:
        # print('last_overlaping: {} bar_before: {}'.format(last_overlaping,bar_before))
        range_starts.append(bar_before)
        bar_before=last_overlaping
    else:
        bar_before+=1
        # print('bar_before: ', bar_before)

overlaping_count+=len(range_starts)
print('overlaping_count: ', overlaping_count)
print('range_starts: ', range_starts)

last_overlaping: 4 bar_before: 0
last_overlaping: 10 bar_before: 6
overlaping_count:  8
range_starts:  [0, 6]


In [12]:
li=[(1,2),(1,2),(1,2)]
pd.DataFrame(li,columns=["first", "second"])

,first,second
0,1,2
1,1,2
2,1,2
